## Lambda Layer
- Lambda(lambda x : function)의 형식
- x 는 이전 레이어 전체를 말한다. 
- 만약 element wise 를 하고 싶으면 아래처럼 backend.map_fn 을 이용하면 된다. 

In [7]:
'''
def inverse(x):
    return x * -1.
rel2 = layers.Lambda(lambda x : K.map_fn(inverse ,x ))(l) 
rel3 = layers.Lambda(lambda x : K.map_fn(lambda y : y * -1 ,x ))(l) 
'''

'\ndef inverse(x):\n    return x * -1.\nrel2 = layers.Lambda(lambda x : K.map_fn(inverse ,x ))(l) \nrel3 = layers.Lambda(lambda x : K.map_fn(lambda y : y * -1 ,x ))(l) \n'

In [1]:
from keras import layers
from keras import models , optimizers , losses
from keras import initializers
import numpy as np
from keras import backend as K

C:\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
x = [[1,2,3],[4,5,6]]
xx =[[10,20,30],[40,50,60]]

x = np.asarray(x)
xx = np.asarray(xx)
y = [0,1]


In [6]:
# 모든 엘리먼트들에게 -1을 곱하는 레이어
def inverse(x):
    return x * -1.



input1 = layers.Input(shape = (3,) )
l = layers.Dense( 1 , kernel_initializer = initializers.RandomUniform(0,1) )(input1)

rel2 = layers.Lambda(lambda x : K.map_fn(inverse ,x ))(l) 
rel3 = layers.Lambda(lambda x : K.map_fn(lambda y : y * -1 ,x ))(l) 


model = models.Model(inputs = input1 , outputs = [l,rel2,rel3])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#res = model1.fit(x,y,epochs = 2 , )

res = model.predict(xx)

print("result")
print(res[0] , res[1] , res[2])


result
[[28.973614]
 [80.5121  ]] [[-28.973614]
 [-80.5121  ]] [[-28.973614]
 [-80.5121  ]]


In [4]:
# 모든 엘리먼트들에게 -1을 곱하고, 다른 레이어 결과를 빼는 방법
def inverse(x):
    return x * -1.

input1 = layers.Input(shape = (3,) )
input2 = layers.Input(shape = (3,) )
input3 = layers.Input(shape = (3,))

l = layers.Dense( 1 , kernel_initializer = initializers.RandomUniform(0,1) )
rel2 = layers.Lambda(lambda x : K.map_fn(inverse ,x )) 

output1 = l(input1)
output2 = rel2(l(input2))
output3 = l(input2)

using_add = layers.add([output1,output2])
using_substract = layers.subtract([output1,output3])


model = models.Model(inputs = [input1,input2,input3] , outputs = [using_add,using_substract])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#res = model1.fit(x,y,epochs = 2 , )

res = model.predict([x,xx,x])

print("result")
print(res[0])
print(res[1])



result
[[-30.283611]
 [-67.67304 ]]
[[-30.283611]
 [-67.67304 ]]


In [5]:
# Lambda 레이어에 기존 빌트인 레이어 섞는 방법
# 모든 엘리먼트들에게 -1을 곱하는 레이어
def inverse(x):
    return x * -1.

input1 = layers.Input(shape = (3,) )
l = layers.Dense( 2 , kernel_initializer = initializers.RandomUniform(0,1) )(input1)

rel2 = layers.Lambda(lambda x : K.map_fn(inverse ,x ))(l) 

# rel2 의 결과는 [a,b] 형식인데 여기에 10*(a+b) 가 결과값으로 나온다. 
rel3 = layers.Lambda(lambda x : layers.Dense( 1 , kernel_initializer = initializers.Constant(10) )(x))(l) 


model = models.Model(inputs = input1 , outputs = [l,rel2,rel3])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#res = model1.fit(x,y,epochs = 2 , )

res = model.predict(xx)

print("result")
print(res[0] , res[1] , res[2])


result
[[23.463171 33.40927 ]
 [59.93864  75.5981  ]] [[-23.463171 -33.40927 ]
 [-59.93864  -75.5981  ]] [[ 568.7244]
 [1355.3674]]
